<a href="https://colab.research.google.com/github/SokolovVladimir/dl_engineer_karpov_course/blob/main/hw_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 2

В этом задании:

1. Сделаем регрессию над данными через scikit-learn: сначала через регресию, потом через бустинг.
2. Сравним результаты с константным предсказанием.
3. Сделаем нейронную сеть на полносвязных слоях, обучим над теми же данными - и сравним с лин. регрессией и бустингом.



In [1]:
import pandas as pd
import numpy as np
import random

# Для воспроизводимости
seed = 0
np.random.seed(seed)
random.seed(seed)

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [3]:
!pip install torch==2.6.0+cu124

ERROR: Could not find a version that satisfies the requirement torch==2.6.0+cu124 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==2.6.0+cu124


In [34]:
# torch                              2.6.0+cu124
# numpy                              2.0.2
# scikit-learn                       1.6.1
# pandas
# albumentations                     2.0.5
# torchvision                        0.21.0+cu124

#torch==2.2.1
#numpy==1.24.4
#scikit-learn==1.3.2
#pandas==2.0.3
#albumentations==1.4.2
#torchvision==0.17.1
#tqdm==4.66.2

In [2]:
csv_url = "https://raw.githubusercontent.com/SokolovVladimir/dl_engineer_karpov_course/refs/heads/main/insurance.csv"

In [3]:
df = pd.read_csv(csv_url)

### Задание №1:
Cделайте train/test split на данных в пропорции 0.8/0.2, залейте в лмс код, который в `df_train`, `df_test`
сохранит датафрейм с тренировочными и тестовыми данными соответственно.

P.S Использовать train_test_split из scikit-learn запрещено - разбивайте вручную через индексы.

In [4]:
idx_test = np.random.randint(0, df.shape[0], int(0.2 * df.shape[0]))
df_test = df.iloc[idx_test]
df_train = df.drop(index=idx_test)

#df_train = df.iloc[0:round(df_lenght*0.8)]
#df_test = df.iloc[round(df_lenght*0.8):]

### Задание №2:
Сделайте `OHE` на колонки sex, region, smoker.

Нужно сделать как на train, так и на test датасете.
Считайте, что исходные датафреймы сохранены в `df_train` и `df_test`.
При этом в `df_train`, `df_test` должны остаться старые колонки - т.е. его нужно обогатить.


In [5]:
df_train = pd.get_dummies(df_train, columns=["sex", "region", "smoker"])
df_test = pd.get_dummies(df_test, columns=["sex", "region", "smoker"])

### Задание №4:
Нормализуйте колонки, которые вы отметили в квизе.
Считайте, что исходные датафреймы сохранены в `df_train` и `df_test`.

Сдайте код, который модифицирует `df_train` и `df_test` так, чтобы численные колонки из прошлого пункта стали нормированы.

In [6]:
num_cols = ["age", "bmi", "charges"]
train_mean = df_train[num_cols].mean()
train_std = df_train[num_cols].std()
df_train[num_cols] = (df_train[num_cols] - train_mean) / train_std
df_test[num_cols] = (df_test[num_cols] - train_mean) / train_std

### Задание №5
Реализуйте функцию, считающую `MSE` метрику.

Ваша функция должна уметь принимать torch.Tensor, numpy-массивы и pd.Series.

In [7]:
def metric(preds, y):
    return ((preds - y) ** 2).mean()

### Задание №6
Реализуйте бейзлайн на `LinearRegression` и `GradientBoostingRegressor`, отправьте метрики на _тестовой выборке_ в ЛМС.

Используйте гиперпараметры по-умолчанию в обоих моделях.

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()
gb_model = GradientBoostingRegressor()

In [9]:
y_train = df_train.pop("charges")
y_test = df_test.pop("charges")

In [10]:
linear_reg_model.fit(df_train, y_train)

LinearRegression()

In [11]:
test_pred_linreg = linear_reg_model.predict(df_test)


In [12]:
mse_linreg = metric(test_pred_linreg, y_test)
print(mse_linreg)

0.32951219663726294


In [13]:
gb_model.fit(df_train, y_train)

GradientBoostingRegressor()

In [14]:
test_pred_gb = gb_model.predict(df_test)
mse_gb = metric(test_pred_gb, y_test)
print(mse_gb)

0.21278812796407792


### Задание №7
Вычислите среднее значение целевой переменной на тренировочной выборке (train).

Подсчитайте MSE при константном предсказании этим средним и отправьте его в ЛМС.

In [15]:
train_mean = y_train.mean()
mse_mean = metric(train_mean, y_test)
print(mse_mean)

1.1045869096529346


### Задание №8
Создайте сеть, состоящую из одного слоя Linear, залейте в лмс код, который в `model` запишет вашу модель

В качестве признаков используйте все колонки в текущем датасете, за исключением таргета

In [16]:
import torch.nn as nn


def build_model():
  return nn.Linear(11, 1)


model = build_model()

In [17]:
import torch

torch.random.manual_seed(seed)
t_x_train = torch.from_numpy(df_train.to_numpy().astype(float)).to(dtype=torch.float32)
t_y_train = torch.from_numpy(y_train.to_numpy()).to(dtype=torch.float32)


### Задание №9
Напишите функцию `train_loop`, которая будет учить модель по данным на 2к итераций.
Считайте, что данные уже хранятся в переменных `t_x_train`, `t_y_train`.
Ваша функция `train_loop` должна вернуть список из лоссов на каждой итерации (т.е. список длины 2000).

Используйте `learning_rate=1e-2` в оптимизаторе.

Для простоты за одну итерацию делайте проход вперед и проход назад на всех наших обучающих данных.
Это будет полный градиентный спуск (не по батчам) - можем себе позволить, данных немного.

_Подсказка 1_: Вам не обязательно учить модель на видеокарте, CPU будет достаточно.

_Подсказка 2_: `tqdm` - это библиотека, которая рисует прогресс итераций

In [18]:
import tqdm
from torch.optim.sgd import SGD

epochs = 2000
learning_rate = 1e-2
optimizer = SGD(model.parameters(), lr=learning_rate)

def train_loop(model: nn.Module) -> list[float]:
    losses = []
    optim = SGD(model.parameters(), lr=1e-2)
    #criterion = nn.MSELoss()
    for _ in tqdm.trange(2000):
        optim.zero_grad()
        out = model(t_x_train)[:, 0]
        #loss = criterion(out, t_y_train)
        loss = metric(out, t_y_train)
        loss.backward()
        losses.append(loss.item())
        optim.step()

    return losses

In [19]:
losses = train_loop(model)

100%|██████████| 2000/2000 [00:01<00:00, 1705.41it/s]


### Задание №10
Обучите модель, состоящую из одного слоя `Linear`.
Приложите в ЛМС метрику `MSE` на тестовых данных.
Используйте `learning_rate=1e-2` в оптимизаторе.

Когда будете тестировать, не забудьте перенести тестовые данные в `torch.Tensor`

In [20]:
t_x_test = torch.from_numpy(df_test.to_numpy().astype(float)).to(dtype=torch.float32)
t_y_test = torch.from_numpy(y_test.to_numpy()).to(dtype=torch.float32)

In [22]:
with torch.no_grad():
  predict_test_linear = model(t_x_test)[:, 0]
#criterion = nn.MSELoss()
mse_linear_model = metric(predict_test_linear, t_y_test)
mse_linear_model.item()

0.3295111060142517

### Задание №11
Вам необходимо усложнить существующую нейронную сеть, добавив один скрытый слой.

Используйте следующие параметры:

Размерность скрытого слоя: 6 нейронов, функция активации -  `ReLU`

Приложите в лмс код, который в переменную `model` запишет вашу модель

In [26]:
def build_model():
  return nn.Sequential(
      nn.Linear(11, 6),
      nn.ReLU(),
      nn.Linear(6, 1),
  )


model_with_hide = build_model()

### Задание №12
Приложите в ЛМС метрику качества этой сети после 2к итераций обучения.
Эту модель можно обучить на CPU, не обязательно на видеокарте.

Используйте для обучения ту же функцию `train_loop` с теми же параметрами (`learning rate`, число итераций и т.п.)

In [27]:
losses = train_loop(model_with_hide)

100%|██████████| 2000/2000 [00:01<00:00, 1211.47it/s]


In [28]:
with torch.no_grad():
  predict_test_linear_hide = model_with_hide(t_x_test)[:, 0]
#criterion = nn.MSELoss()
mse_linear_model_hide = metric(predict_test_linear_hide, t_y_test)
mse_linear_model_hide.item()

0.24223779141902924

### Задание №13
Добавьте дополнительные слои в нейронную сеть

Вам необходимо усложнить нейронную сеть, добавив еще 2-3 скрытых слоя с такими же размерностями, как в предыдущем задании.

Приложите в лмс код, который в переменную `model` запишет вашу модель

In [ ]:
def build_model(): ...


model = build_model()

### Задание №14
Приложите в ЛМС метрику качества после 2к итераций обучения.

Используйте для обучения ту же функцию `train_loop` с теми же параметрами (`learning rate`, число итераций и т.п.)

Эту модель можно обучить на CPU, не обязательно на видеокарте.